<a href="https://colab.research.google.com/github/vasan-rj/python-youtube-code/blob/master/url_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import itertools
from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from lightgbm import LGBMClassifier
import os
import seaborn as sns
from wordcloud import WordCloud

In [3]:
from math import log
from re import compile
from urllib.parse import urlparse
from socket import gethostbyname
# from pyquery import PyQuery
from requests import get
from json import dump
from string import ascii_lowercase
from numpy import array, iterable
import pandas as pd
from time import time

In [2]:
from google.colab import files
data=files.upload()

Saving malicious_phish.csv to malicious_phish.csv


In [4]:
df=pd.read_csv('malicious_phish.csv')
print(df.shape)
df.head()

(651191, 2)


,url,type
0,br-icloud.com.br,phishing
1,mp3raid.com/music/krizz_kaliko.html,benign
2,bopsecrets.org/rexroth/cr/1.htm,benign
3,http://www.garage-pirenne.be/index.php?option=...,defacement
4,http://adventure-nicaragua.net/index.php?optio...,defacement


In [9]:
df_vvv=pd.read_csv('malicious_phish.csv')

In [5]:
df.type.value_counts()

benign        428103
defacement     96457
phishing       94111
malware        32520
Name: type, dtype: int64

In [6]:
df_phish = df[df.type=='phishing']
df_malware = df[df.type=='malware']
df_deface = df[df.type=='defacement']
df_benign = df[df.type=='benign']

url     94111
type    94111
dtype: int64

In [10]:
import re
#Use of IP or not in domain
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0
df['use_of_ip'] = df['url'].apply(lambda i: having_ip_address(i))

In [21]:
from urllib.parse import urlparse

def abnormal_url(url):
    hostname = urlparse(url).hostname
    hostname = str(hostname)
    match = re.search(hostname, url)
    if match:
        # print match.group()
        return 1
    else:
        # print 'No matching pattern found'
        return 0


df['abnormal_url'] = df['url'].apply(lambda i: abnormal_url(i))

In [30]:
df['abnormal_url'].value_counts()[1]

188006

In [32]:
df

,url,type,use_of_ip,abnormal_url
0,br-icloud.com.br,phishing,0,0
1,mp3raid.com/music/krizz_kaliko.html,benign,0,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,0,1
4,http://adventure-nicaragua.net/index.php?optio...,defacement,0,1
...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,0,0
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,0,0
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,0,0
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,0,0


In [33]:
from googlesearch import search

In [41]:
def count_www(url):
    url.count('www')
    return url.count('www')

df['count-www'] = df['url'].apply(lambda i: count_www(i))

def count_atrate(url):

    return url.count('@')

df['count@'] = df['url'].apply(lambda i: count_atrate(i))


def no_of_dir(url):
    urldir = urlparse(url).path
    return urldir.count('/')

df['count_dir'] = df['url'].apply(lambda i: no_of_dir(i))

def no_of_embed(url):
    urldir = urlparse(url).path
    return urldir.count('//')

df['count_embed_domian'] = df['url'].apply(lambda i: no_of_embed(i))


def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return 1
    else:
        return 0


df['short_url'] = df['url'].apply(lambda i: shortening_service(i))

In [42]:
df

,url,type,use_of_ip,abnormal_url,count-www,count@,count_dir,count_embed_domian,short_url
0,br-icloud.com.br,phishing,0,0,0,0,0,0,0
1,mp3raid.com/music/krizz_kaliko.html,benign,0,0,0,0,2,0,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,0,0,0,3,0,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,0,1,1,0,1,0,0
4,http://adventure-nicaragua.net/index.php?optio...,defacement,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,0,0,0,0,3,0,0
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,0,0,0,0,4,0,1
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,0,0,1,0,4,0,1
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,0,0,0,0,2,0,0


In [43]:
df['short_url'].value_counts()[1]

39755

In [44]:
def count_https(url):
    return url.count('https')

df['count-https'] = df['url'].apply(lambda i : count_https(i))

def count_http(url):
    return url.count('http')

df['count-http'] = df['url'].apply(lambda i : count_http(i))

In [47]:
df['count-https'].value_counts()[1]
#df['count-http'].value_counts()[1]

16197

In [49]:
def count_per(url):
    return url.count('%')

df['count%'] = df['url'].apply(lambda i : count_per(i))

def count_ques(url):
    return url.count('?')

df['count?'] = df['url'].apply(lambda i: count_ques(i))

def count_hyphen(url):
    return url.count('-')

df['count-'] = df['url'].apply(lambda i: count_hyphen(i))

def count_equal(url):
    return url.count('=')

df['count='] = df['url'].apply(lambda i: count_equal(i))

def url_length(url):
    return len(str(url))


#Length of URL
df['url_length'] = df['url'].apply(lambda i: url_length(i))
#Hostname Length

def hostname_length(url):
    return len(urlparse(url).netloc)

df['hostname_length'] = df['url'].apply(lambda i: hostname_length(i))

df.head()

def suspicious_words(url):
    match = re.search('PayPal|login|signin|bank|account|update|free|lucky|service|bonus|ebayisapi|webscr',
                      url)
    if match:
        return 1
    else:
        return 0
df['sus_url'] = df['url'].apply(lambda i: suspicious_words(i))


def digit_count(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits


df['count-digits']= df['url'].apply(lambda i: digit_count(i))


def letter_count(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters


df['count-letters']= df['url'].apply(lambda i: letter_count(i))

In [50]:
df

,url,type,use_of_ip,abnormal_url,count-www,count@,count_dir,count_embed_domian,short_url,count-https,count-http,count%,count?,count-,count=,url_length,hostname_length,sus_url,count-digits,count-letters
0,br-icloud.com.br,phishing,0,0,0,0,0,0,0,0,0,0,0,1,0,16,0,0,0,13
1,mp3raid.com/music/krizz_kaliko.html,benign,0,0,0,0,2,0,0,0,0,0,0,0,0,35,0,0,1,29
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,0,0,0,3,0,0,0,0,0,0,0,0,31,0,0,1,25
3,http://www.garage-pirenne.be/index.php?option=...,defacement,0,1,1,0,1,0,0,0,1,0,1,1,4,88,21,0,7,63
4,http://adventure-nicaragua.net/index.php?optio...,defacement,0,1,0,0,1,0,0,0,1,0,1,1,3,235,23,0,22,199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,0,0,0,0,3,0,0,0,0,0,0,0,0,39,0,0,12,21
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,0,0,0,0,4,0,1,0,0,0,0,2,0,44,0,0,7,29
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,0,0,1,0,4,0,1,0,0,0,0,0,0,42,0,0,3,33
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,0,0,0,0,2,0,0,0,0,0,0,0,0,45,0,0,0,36


In [55]:
df['sus_url'].value_counts()[1]

43373

In [59]:
from tld import get_tld

In [57]:
pip install tld

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 1.9 MB/s eta 0:00:00


In [68]:
from urllib.parse import urlparse
from tld import get_tld
import os.path

#First Directory Length
def fd_length(url):
    urlpath= urlparse(url).path
    try:
        return len(urlpath.split('/')[1])
    except:
        return 0

df['fd_length'] = df['url'].apply(lambda i: fd_length(i))

#Length of Top Level Domain
df['tld'] = df['url'].apply(lambda i: get_tld(i,fail_silently=True))


def tld_length(tld):
    try:
        return len(tld)
    except:
        return -1

# df_vvv['tld_length'] = df['tld'].apply(lambda i: tld_length(i))

In [74]:
df

,url,type,use_of_ip,abnormal_url,count-www,count@,count_dir,count_embed_domian,short_url,count-https,...,count?,count-,count=,url_length,hostname_length,sus_url,count-digits,count-letters,fd_length,tld
0,br-icloud.com.br,phishing,0,0,0,0,0,0,0,0,...,0,1,0,16,0,0,0,13,0,None
1,mp3raid.com/music/krizz_kaliko.html,benign,0,0,0,0,2,0,0,0,...,0,0,0,35,0,0,1,29,5,None
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,0,0,0,3,0,0,0,...,0,0,0,31,0,0,1,25,7,None
3,http://www.garage-pirenne.be/index.php?option=...,defacement,0,1,1,0,1,0,0,0,...,1,1,4,88,21,0,7,63,9,be
4,http://adventure-nicaragua.net/index.php?optio...,defacement,0,1,0,0,1,0,0,0,...,1,1,3,235,23,0,22,199,9,net
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,0,0,0,0,3,0,0,0,...,0,0,0,39,0,0,12,21,7,None
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,0,0,0,0,4,0,1,0,...,0,2,0,44,0,0,7,29,8,None
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,0,0,1,0,4,0,1,0,...,0,0,0,42,0,0,3,33,7,None
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,0,0,0,0,2,0,0,0,...,0,0,0,45,0,0,0,36,4,None


In [78]:
df['tld'].value_counts()

com             71688
de              11331
net              9900
nl               6157
com.br           5925
                ...  
mywire.org          1
com.bt              1
uk.com              1
is-slick.com        1
ht                  1
Name: tld, Length: 548, dtype: int64

In [85]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df_vvv["tld_typex"] = lb_make.fit_transform(df["tld"])

In [98]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df["type_code"] = lb_make.fit_transform(df["type"])
# df_vvv["type_code"].value_counts()

In [103]:
 df["type_code"].value_counts()

0    428103
1     96457
3     94111
2     32520
Name: type_code, dtype: int64

In [104]:
df["type_code"]

0         3
1         0
2         0
3         1
4         1
         ..
651186    3
651187    3
651188    3
651189    3
651190    3
Name: type_code, Length: 651191, dtype: int64

In [71]:
import seaborn as sns

In [100]:
df

,url,type,use_of_ip,abnormal_url,count-www,count@,count_dir,count_embed_domian,short_url,count-https,...,count-,count=,url_length,hostname_length,sus_url,count-digits,count-letters,fd_length,tld,type_code
0,br-icloud.com.br,phishing,0,0,0,0,0,0,0,0,...,1,0,16,0,0,0,13,0,None,3
1,mp3raid.com/music/krizz_kaliko.html,benign,0,0,0,0,2,0,0,0,...,0,0,35,0,0,1,29,5,None,0
2,bopsecrets.org/rexroth/cr/1.htm,benign,0,0,0,0,3,0,0,0,...,0,0,31,0,0,1,25,7,None,0
3,http://www.garage-pirenne.be/index.php?option=...,defacement,0,1,1,0,1,0,0,0,...,1,4,88,21,0,7,63,9,be,1
4,http://adventure-nicaragua.net/index.php?optio...,defacement,0,1,0,0,1,0,0,0,...,1,3,235,23,0,22,199,9,net,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651186,xbox360.ign.com/objects/850/850402.html,phishing,0,0,0,0,3,0,0,0,...,0,0,39,0,0,12,21,7,None,3
651187,games.teamxbox.com/xbox-360/1860/Dead-Space/,phishing,0,0,0,0,4,0,1,0,...,2,0,44,0,0,7,29,8,None,3
651188,www.gamespot.com/xbox360/action/deadspace/,phishing,0,0,1,0,4,0,1,0,...,0,0,42,0,0,3,33,7,None,3
651189,en.wikipedia.org/wiki/Dead_Space_(video_game),phishing,0,0,0,0,2,0,0,0,...,0,0,45,0,0,0,36,4,None,3


*host based  extraction*

In [111]:
pip install whois

In [108]:
pip install waybackpy

In [109]:
from whois import whois
from waybackpy import Cdx
from socket import gethostbyname
from shodan import Shodan
from requests import get
from urllib.parse import urlparse
from datetime import datetime
from re import compile
from json import dump, loads
from time import sleep

ImportError: ignored

In [115]:
df_vvv['url_agex'].value_counts()

Series([], Name: url_age, dtype: int64)